# Tutorial: Grover's searching algorithm

QuICT includes implementation of the most basic (partial) Grover search algorithm.[1][2]

## Algorithm

The overall circuit for Grover search which uses $\lceil\frac{\pi}{4}\sqrt{N}\rceil$ oracle calls for large $N$:

<center>
<img src='./basic_Grover_circuit.png'>
</center>

Phase part which uses $O(n)=O(\log N)$ elementary gates (with MCT implemented in $O(n)$ elementary gates and 1 ancillary qubit[5]):

<center>
<img src='./phase_circuit.png'>
</center>

Partial Grover search use local Grover operation along with global Grover iteration (more specifically, form of input oracle is limited to ensure a xor-oracle can be easily constructed based on the input oracle):

<center>
<img src='./partial_Grover_circuit.png'>
</center>

And local Grover operation is shown below:

<center>
<img src='./local_g_iter_circuit.png'>
</center>

## Example: binary sodoku

Consider solving a 2×2 binary sudoku that

- No column may contain the same value twice
- No row may contain the same value twice

<center>
<img src='./binary_sudoku.png'>
</center>

i.e. 

$$f(x)=[(x_0\oplus x_1) \& (x_2\oplus x_3) \& (x_0\oplus x_2) \& (x_1\oplus x_3)]$$

To start with, we construct the oracle $O\ket{x}=(-1)^{f(x)}\ket{x}$:

In [ ]:
from QuICT.algorithm.quantum_algorithm.grover import Grover,PartialGrover
from QuICT.qcda.synthesis.mct import MCTOneAux

def sudoku_oracle():
    clauses_list = [[0,1],[2,3],[0,2],[1,3]]
    reg_q = list(range(4))
    clause_q = list(range(4,8))
    result_q = [8]
    ancilla_q = [9]
    cgate = CompositeGate()
    with cgate:
        # |-> in result_q
        X & result_q[0]
        H & result_q[0]
        # sentence
        for i in range(len(clauses_list)):
            CX & [reg_q[clauses_list[i][0]],clause_q[i]]
            CX & [reg_q[clauses_list[i][1]],clause_q[i]]
        MCTOneAux.execute(5+1) & (clause_q+result_q+ancilla_q)
        # un-compute
        for i in range(len(clauses_list)):
            CX & [reg_q[clauses_list[i][0]],clause_q[i]]
            CX & [reg_q[clauses_list[i][1]],clause_q[i]]
        H & result_q[0]
        X & result_q[0]
    return 6, cgate

The circuit of the oracle:

In [ ]:
k,cgate = sudoku_oracle()
circ = Circuit(10)
cgate | circ
circ.draw()

Construct Grover search circuit on the oracle and run:

In [ ]:
n = 4
k, oracle = sudoku_oracle()
circ = Grover.circuit(n, k, oracle, m=2)

amp = CircuitSimulator().run(circ)
x = bin(int(circ[list(range(n))]))[2:].rjust(4,'0')[::-1]
print(f'{x[0]}|{x[1]}\n-+-\n{x[2]}|{x[3]}')

Consider another example with only one solution so we can compare `Grover` module and `PartialGrover` module.

There are 4 statements $x_i:\sum_j [x_j]=i, i=1,2,3,4$, and the problem is to decide the truth value of each staatement. Notice that these statements contradict each other therefore at most one of them is correct, then it is easy to check that $(True,True,False,True)$ is the only solution.

More precisely, $f(x)=\land_{i=1,2,3,4}((\neg x_i\land\sum_{j\neq i}[x_j]\neq i) \lor (x_i\land\sum_{j\neq i}[x_j]=i))$. But actually we use $f_a(x)=f(x)\land(\land_{i=4+1...4+a} x_i)$ to distinguish Grover and Partial Grover module.

Again we can construct the oracle and find solution with Grover module:

In [ ]:
def sentence(control_q:int,local_var_q,clause_q:int,ancilla_q):
    assert len(local_var_q)==3 and len(ancilla_q)==2
    cgate = CompositeGate()
    with cgate:
        CX & [local_var_q[0],clause_q]
        CX & [local_var_q[1],clause_q]
        CX & [local_var_q[2],clause_q]
        CCX & [local_var_q[1],local_var_q[2],ancilla_q[0]]
        X & ancilla_q[0]
        CCX & [clause_q,ancilla_q[0],ancilla_q[1]]
        X & ancilla_q[0]
        #uncompute clause_q
        CX & [local_var_q[0],clause_q]
        CX & [local_var_q[1],clause_q]
        CX & [local_var_q[2],clause_q]

        X & ancilla_q[1]
        CCX & [control_q,ancilla_q[1],clause_q]
        X & ancilla_q[1]
        X & clause_q

        #uncompute ancilla_q
        X & ancilla_q[0]
        CCX & [local_var_q[0],ancilla_q[0],ancilla_q[1]]
        CCX & [local_var_q[1],ancilla_q[0],ancilla_q[1]]
        CCX & [local_var_q[2],ancilla_q[0],ancilla_q[1]]
        X & ancilla_q[0]
        CCX & [local_var_q[1],local_var_q[2],ancilla_q[0]]
    return cgate

def sentence_adjoint(control_q:int,local_var_q,clause_q:int,ancilla_q):
    assert len(local_var_q)==3 and len(ancilla_q)==2
    cgate = CompositeGate()
    with cgate:
        #uncompute ancilla_q
        CCX & [local_var_q[1],local_var_q[2],ancilla_q[0]]
        X & ancilla_q[0]
        CCX & [local_var_q[2],ancilla_q[0],ancilla_q[1]]
        CCX & [local_var_q[1],ancilla_q[0],ancilla_q[1]]
        CCX & [local_var_q[0],ancilla_q[0],ancilla_q[1]]
        X & ancilla_q[0]

        X & clause_q
        X & ancilla_q[1]
        CCX & [control_q,ancilla_q[1],clause_q]
        X & ancilla_q[1]

        CX & [local_var_q[2],clause_q]
        CX & [local_var_q[1],clause_q]
        CX & [local_var_q[0],clause_q]
        X & ancilla_q[0]
        CCX & [clause_q,ancilla_q[0],ancilla_q[1]]
        X & ancilla_q[0]
        CCX & [local_var_q[1],local_var_q[2],ancilla_q[0]]
        CX & [local_var_q[2],clause_q]
        CX & [local_var_q[1],clause_q]
        CX & [local_var_q[0],clause_q]
    return cgate

def CCCX(reg_q):
    assert len(reg_q)==5
    cgate = CompositeGate()
    with cgate:
        CCX & [reg_q[0],reg_q[1],reg_q[4]]
        H & [reg_q[3]]
        S & [reg_q[4]]
        CCX & [reg_q[2],reg_q[3],reg_q[4]]
        S_dagger & [reg_q[4]]
        CCX & [reg_q[0],reg_q[1],reg_q[4]]
        S & [reg_q[4]]
        CCX & [reg_q[2],reg_q[3],reg_q[4]]
        H & [reg_q[3]]
        S_dagger & [reg_q[4]]
    return cgate

def puzzle_oracle():
    n_true_var = 4
    n_var = 8
    n_clause = 4
    var_q = list(range(n_var))
    clause_q = list(range(n_var,n_var+n_clause))
    result_q = [n_var+n_clause]
    ancilla_q = [n_var+n_clause+1,n_var+n_clause+2]
    cgate = CompositeGate()
    with cgate:
        # # |-> in result_q
        X & result_q[0]
        H & result_q[0]
        # sentence1
        for i in [1,2,3]:
            X & i
        sentence(0,[1,2,3],clause_q[0],ancilla_q) | cgate
        for i in [1,2,3]:
            X & i
        # sentence2
        sentence(1,[0,2,3],clause_q[1],ancilla_q) | cgate
        # sentence3
        for i in [0,1,3]:
            X & i
        CCCX([0,1,3,clause_q[2],ancilla_q[1]]) | cgate
        X & clause_q[2]
        CCX & [2,clause_q[2],ancilla_q[0]]
        X & clause_q[2]
        CCCX([0,1,3,clause_q[2],ancilla_q[1]]) | cgate
        for i in [0,1,3]:
            X & i
        CX & [ancilla_q[0],clause_q[2]]
        CX & [clause_q[2],ancilla_q[0]]
        X & clause_q[2]
        # sentence4
        for i in [0,1,2,3]:
            X & i
        CCCX([0,1,2,ancilla_q[0],ancilla_q[1]]) | cgate
        X & ancilla_q[0]
        CCX & [3,ancilla_q[0],clause_q[3]]
        X & ancilla_q[0]
        CCCX([0,1,2,ancilla_q[0],ancilla_q[1]]) | cgate
        for i in [0,1,2,3]:
            X & i
        # MCT
        MCTOneAux.execute(n_var-n_true_var+len(clause_q)+2) & (var_q[n_true_var:n_var]+clause_q+result_q+[ancilla_q[0]])
        #uncompute clauses
        # |-> in result_q
        H & result_q[0]
        X & result_q[0]
        # sentence1
        for i in [1,2,3]:
            X & i
        sentence_adjoint(0,[1,2,3],clause_q[0],ancilla_q) | cgate
        for i in [1,2,3]:
            X & i
        # sentence2
        sentence_adjoint(1,[0,2,3],clause_q[1],ancilla_q) | cgate
        # sentence3
        X & clause_q[2]
        CX & [clause_q[2],ancilla_q[0]]
        CX & [ancilla_q[0],clause_q[2]]
        for i in [0,1,3]:
            X & i
        CCCX([0,1,3,clause_q[2],ancilla_q[1]]) | cgate
        X & clause_q[2]
        CCX & [2,clause_q[2],ancilla_q[0]]
        X & clause_q[2]
        CCCX([0,1,3,clause_q[2],ancilla_q[1]]) | cgate
        for i in [0,1,3]:
            X & i
        # sentence4
        for i in [0,1,2,3]:
            X & i
        CCCX([0,1,2,ancilla_q[0],ancilla_q[1]]) | cgate
        X & ancilla_q[0]
        CCX & [3,ancilla_q[0],clause_q[3]]
        X & ancilla_q[0]
        CCCX([0,1,2,ancilla_q[0],ancilla_q[1]]) | cgate
        for i in [0,1,2,3]:
            X & i
    return 7, cgate

In [ ]:
n = 8
k, oracle = puzzle_oracle()
circ = Grover.circuit(n, k, oracle, measure=False)

amp = CircuitSimulator().run(circ)
names = [bin(i)[2:].rjust(n,'0') for i in range(1<<n)]
values = trace_prob(amp,list(range(n))).reshape([1<<n])
print(f"success rate = {sum([values[i] if bin(i)[2:].rjust(n,'0')=='00101111' else 0 for i in range(len(values))])}")

import matplotlib.pyplot as plt
plt.figure(figsize=(30,6))
plt.bar(names, values)
plt.xticks(fontsize=6,rotation=75)
plt.yticks(ticks=[i/10 for i in range(11)])
plt.show()

Construct Partial Grover search circuit on same oracle and run:

In [ ]:
n = 8
n_block = 3
print(f"run with n = {n}, block size = {n_block}")
k, oracle = puzzle_oracle()
circ = PartialGrover.circuit(n, n_block, k, oracle, measure=False)

from QuICT.simulation.cpu_simulator import CircuitSimulator
amp = CircuitSimulator().run(circ)
names = [bin(i)[2:].rjust(n,'0') for i in range(1<<n)]
values = trace_prob(amp,list(range(n))).reshape([1<<n])
print(f"success rate = {sum([values[i] if bin(i)[2:].rjust(n,'0')[:3]=='001' else 0 for i in range(len(values))])}")

import matplotlib.pyplot as plt
plt.figure(figsize=(30,6))
plt.bar(names, values)
plt.xticks(fontsize=6,rotation=75)
plt.yticks(ticks=[i/10 for i in range(11)])
plt.show()

Example above demonstrates the fact that partial search uses fewer Oracle calls and only aims to find block address. A detailed comparison is given below:

| algorithm              | Grover(with unique target)   | Partial Grover                                |
| ---------------------- | ---------------------------- | --------------------------------------------- |
| expected output        | target address $a=a_1...a_n$ | block address of target $a_{block}=a_1...a_k$ |
| oracle calls           | $\approx\pi/4\sqrt{N}$       | $\approx\pi/4\sqrt{N}-0.34\sqrt{N/K}$         |
| other elementary gates | $O(\sqrt{N}\log N)$          | $O(\sqrt{N}\log N)$                           |
| success rate           | $1-O(N^{-1/2})$              | $1-O(N^{-1/4})$                               |

## Example: CNF solver

[Conjunctive Normal Form](https://en.wikipedia.org/wiki/Conjunctive_normal_form)(CNF) is a canonical expression in Boolean logic. Next we constructs the oracle circuit for any CNF formula given its description file, and combine the oracle with `Grover` module to find solutions.

This oracle is implemented in `QuICT`.

In [ ]:
from QuICT.core import Circuit
from QuICT.core.gate import *
from QuICT.core.gate.backend import MCTOneAux
from QuICT.simulation.state_vector import ConstantStateVectorSimulator

In [ ]:
from QuICT.algorithm.quantum_algorithm.CNF import CNFSATOracle

The following function reads information from CNF description file:

In [ ]:
def read_CNF(cnf_file):
        variable_number = 0
        clause_number = 0
        CNF_data = []
        f = open(cnf_file, 'r') 
        for line in f.readlines():
            new = line.strip().split()
            int_new=[]
            if new[0] == 'p':
                variable_number = int(new[2])
                clause_number = int(new[3])
            else:
                for x in new:
                    if (x != '0') and (int(x) not in int_new):
                        int_new.append(int(x))
                        if (- int(x)) in int_new:
                            int_new = []
                            break
            CNF_data.append(int_new)
        f.close()
        return variable_number, clause_number, CNF_data

Checking CNF solution is neccessary since the algorithm is not deterministic: 

In [ ]:
def check_solution(variable_data, variable_number, clause_number, CNF_data):
    cnf_result = 1
    for i in range(clause_number):
        clause_result = 0
        if CNF_data[i+1] == []:
            clause_result = 1
        else:
            for j in range(len(CNF_data[i+1])):
                if CNF_data[i+1][j] > 0:
                    clause_result = clause_result + variable_data[CNF_data[i+1][j]-1]
                else:
                    if CNF_data[i+1][j] < 0:
                        clause_result = clause_result  + ( 1 - variable_data[-CNF_data[i+1][j]-1] )
            if clause_result == 0:
                cnf_result = 0
                break
    if cnf_result == 1:
        return True
    else:
        return False

The combination is as follows, and we try to find solution to a 16-variable CNF formula in `test.cnf`:

In [ ]:
file_path = 'test.cnf'
variable_number, clause_number, CNF_data = read_CNF(file_path)

ancilla_qubits_num=5
dirty_ancilla=1
cnf = CNFSATOracle()
cnf.run(file_path, ancilla_qubits_num, dirty_ancilla)

oracle = cnf.circuit()
grover = Grover(ConstantStateVectorSimulator())
    
circ = grover.circuit(variable_number, ancilla_qubits_num + dirty_ancilla, oracle, n_solution=1, measure=False, is_bit_flip=True)
print(f"constrcution finished.")
grover.simulator.run(circ)
print(f"simulation   finished.")
n_hit = 0
n_all = 1000
result_samples = grover.simulator.sample(n_all)
print(f"sampling     finished.")

We can check the success rate:

In [ ]:
result_var_samples = np.array(result_samples).reshape((1<<variable_number,1<<(ancilla_qubits_num + dirty_ancilla))).sum(axis=1)
for result in range(1<<variable_number):
    result_str = bin(result)[2:].rjust(variable_number,'0')
    if check_solution([int(x) for x in result_str], variable_number, clause_number, CNF_data):
        n_hit += result_var_samples[result]
print(f"[{n_hit}/{n_all}]:{n_hit/n_all:.3f}")